In [1]:
from transformers import CLIPModel, CLIPProcessor, CLIPConfig, CLIPTextConfig, CLIPVisionConfig
from transformers import pipeline
from PIL import Image
import requests

/home/antony/Documents/Visual Studio Code/clip/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
CLIPTextConfig()

CLIPTextConfig {
  "attention_dropout": 0.0,
  "bos_token_id": 49406,
  "eos_token_id": 49407,
  "hidden_act": "quick_gelu",
  "hidden_size": 512,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 77,
  "model_type": "clip_text_model",
  "num_attention_heads": 8,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "projection_dim": 512,
  "transformers_version": "4.41.2",
  "vocab_size": 49408
}

In [17]:
clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [7]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [8]:
inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

In [11]:
outputs = clip(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

In [45]:
from transformers import AutoImageProcessor, AutoTokenizer, AutoBackbone, AutoProcessor, AutoModel, AutoFeatureExtractor, CLIPVisionConfig, CLIPTokenizer, CLIPTokenizerFast
from transformers import PreTrainedTokenizerFast, PreTrainedTokenizer, CLIPImageProcessor

In [18]:
clip:CLIPModel = CLIPModel(CLIPConfig())
tok:CLIPTokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
tok_fast:CLIPTokenizerFast = CLIPTokenizerFast.from_pretrained("openai/clip-vit-base-patch32")

In [46]:
vis = CLIPImageProcessor()
img = Image.open("./image.jpg")

In [65]:
text = ["a photo of a cat", "a photo of a dog"]
tex_tok = tok(text=text, return_tensors="pt")
tex_tok_fast = tok_fast(text=text, return_tensors="pt")
print(tex_tok)
tex_tok_fast

{'input_ids': tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}


{'input_ids': tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}

In [27]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [29]:
inputs = processor(text=text, images=img, return_tensors="pt")

In [32]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])

In [56]:
vis_img = vis(img, return_tensors="pt")

In [59]:
vis_img['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [66]:
print(tex_tok['input_ids'].shape)
tex_tok['attention_mask'].shape

torch.Size([2, 7])


torch.Size([2, 7])

In [62]:
print(inputs['input_ids'].shape)
print(inputs['attention_mask'].shape)
inputs['pixel_values'].shape

torch.Size([2, 7])
torch.Size([2, 7])


torch.Size([1, 3, 224, 224])

SyntaxError: invalid syntax (2240957684.py, line 1)

In [67]:
outputs = clip(vis_img, tex_tok)
outputs

AttributeError: 